# 3. 추천 시스템 모델링

- TF-IDF로 문장-단어 간 유사도 측정

- TF-IDF벡터를 기반으로, 코사인 유사도를 구한다.

In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/mvs_final_csvfile.csv")

df = data.drop("Unnamed: 0", axis=1 )

In [ ]:
df.head(10)

,title,year,genres,keywords
0,#살아있다,2020,스릴러,좀비 탈출
1,007 노 타임 투 다이,2020,액션,007 첩보원 스파이 본드걸
2,광주비디오 – 사라진 4시간,2020,다큐멘터리,5 18민주화운동 역사 광주
3,국제수사,2020,액션,형사 여행 필리핀
4,그레텔과 헨젤,2020,공포,동화원작 남매 인육
5,나쁜 녀석들 : 포에버,2020,액션,형사 마약 버디무비
6,다만 악에서 구하소서,2020,액션,청부살인업자 복수
7,닥터 두리틀,2020,코미디,초능력 동물
8,딥워터,2020,액션,바다 스쿠버 탈출 자매
9,반도,2020,액션,좀비 군인 탈출


In [ ]:
df.isnull().sum()

title       0
year        0
genres      0
keywords    0
dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf_matrics(column):
    tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
    tfidf_mat = tfidf_vec.fit_transform(column.values.astype("U"))
    return tfidf_mat

tf_idf_mat = tf_idf_matrics(df["keywords"])
print(tf_idf_mat)

  (0, 12728)	0.7149139875639746
  (0, 14272)	0.5233253691708918
  (0, 12710)	0.4637117082493465
  (1, 7802)	0.4753011628765531
  (1, 13371)	0.34361149375750044
  (1, 3)	0.4753011628765531
  (1, 5695)	0.3616475878293956
  (1, 7787)	0.2857528836992443
  (1, 13362)	0.3020481185121082
  (1, 0)	0.3555727721186944
  (2, 9340)	0.47351064654679925
  (2, 24)	0.47351064654679925
  (2, 1454)	0.4174084282330683
  (2, 9339)	0.47351064654679925
  (2, 20)	0.391322382263647
  (3, 9320)	0.509481356481166
  (3, 15696)	0.509481356481166
  (3, 15180)	0.509481356481166
  (3, 9297)	0.3337278534567415
  (3, 15678)	0.33152973022262816
  (4, 2352)	0.5120092586136362
  (4, 3393)	0.5120092586136362
  (4, 11062)	0.37479674330601664
  (4, 2346)	0.4007495814852775
  (4, 3390)	0.4178758336329205
  :	:
  (7821, 3072)	0.4597699694550147
  (7822, 2356)	1.0
  (7823, 4554)	0.4711939770450506
  (7823, 16229)	0.4711939770450506
  (7823, 4552)	0.38940782208437086
  (7823, 5250)	0.3370860689098439
  (7823, 5238)	0.3018626891

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_sim_matrics(column):
    genres_sim = cosine_similarity(column, column)
    sim_idx = np.argsort(-genres_sim)
    return genres_sim, sim_idx

genres_sim, sim_idx = cosine_sim_matrics(tf_idf_mat)

print(genres_sim[:1], "\n",sim_idx)

[[1. 0. 0. ... 0. 0. 0.]] 
 [[   0  172 6867 ... 2614 2645 7827]
 [   1 1344 4156 ... 2618 2646 7827]
 [   2  173 5347 ... 2603 2601 7827]
 ...
 [7825 7081 7785 ... 2600 2598 7827]
 [7826 5225 5224 ... 2600 2598 7827]
 [7827 7760 7129 ... 2603 2601 3913]]


In [ ]:
def recommandation_mvs_top10(inputs_mv, sim_idx):
#검색한 영화의 인덱스를 찾아준다.
    mv_idx = df[df.title==inputs_mv].index.values
    print(mv_idx)

#인덱스 값을 기준으로 유사도가 높은 영화 인덱스를 출력한다.
    sim_mv = sim_idx[mv_idx, :int(10)]
    print(sim_mv)

    sim_mv_idx = sim_mv.reshape(-1) #2차원 벡터 -> 1차원으로 변경
    print(sim_mv_idx)
    
    top_10_title = pd.Series(df.iloc[sim_mv_idx, 0])
    top_10_genres = pd.Series(df.iloc[sim_mv_idx, 2])
    top_10_keywords = pd.Series(df.iloc[sim_mv_idx, 3])
    top_10_years = pd.Series(df.iloc[sim_mv_idx, 1])

    recommandation_df = pd.DataFrame({
        "title": top_10_title,
        "years": top_10_years,
        "genres": top_10_genres,
        "keywords": top_10_keywords
    })
    
    return recommandation_df

inputs_mv = input().strip()
r = recommandation_mvs_top10(inputs_mv, sim_idx)
print(r)

반도
[]
[]
[]
Empty DataFrame
Columns: [title, years, genres, keywords]
Index: []


# functional 하게 코드 구성하기

In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def tf_idf_matrics(column):  #tf-idf사용해서 문장-단어 간 유사도 측정 함수
    tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
    tfidf_mat = tfidf_vec.fit_transform(column.values.astype("U"))
    return tfidf_mat

# tf_idf_mat = tf_idf_matrics(df["keywords"])
# print(tf_idf_mat)

def cosine_sim_matrics(tfidf_mat): #코사인 유사도 측정 함수
    cos_sim = cosine_similarity(tfidf_mat, tfidf_mat)
    sim_idx = np.argsort(-cos_sim)
    return sim_idx

# genres_sim, sim_idx = cosine_sim_matrics(tf_idf_mat)
# print(genres_sim, sim_idx)

def recommandation_mvs_top10(inputs_mv, sim_idx, df): #input을 사용자로부터 받고, 유사도가 제일 높은 10개의 영화를 추천해주는 함수
#검색한 영화의 인덱스를 찾아준다.
    mv_idx = df[df.title==inputs_mv.lstrip().rstrip()].index.values
    print(mv_idx)
    
#인덱스 값을 기준으로 유사도가 높은 영화 인덱스를 출력한다.
    sim_mv = sim_idx[mv_idx, :int(10)]
    print(sim_mv)

    sim_mv_idx = sim_mv.reshape(-1) #2차원 벡터 -> 1차원으로 변경
    print(sim_mv_idx)
    #x = df.iloc[sim_mv_idx]
    
    top_10_title = pd.Series(df.iloc[sim_mv_idx, 0])
    top_10_years = pd.Series(df.iloc[sim_mv_idx, 1])
    top_10_genres = pd.Series(df.iloc[sim_mv_idx, 2])
    top_10_keywords = pd.Series(df.iloc[sim_mv_idx, 3])

    recommandation_df = pd.DataFrame({
        "title": top_10_title,
        "year": top_10_years,
        "genres": top_10_genres,
        "keywords": top_10_keywords
    })
    
    return recommandation_df

# inputs_mv = input()
# r = recommandation_mvs_top10(inputs_mv, sim_idx)
# print(r)

# def main1():
#     drive.mount("/content/gdrive")
#     df = pd.read_csv("/content/gdrive/My Drive/광인사 자연어 프로젝트/final_prepro_data2.csv")
#     tf_idf_mat = tf_idf_matrics(df["keywords"])
#     print(f'TF-IDF-VECTOR- \n {tf_idf_mat}')
#     genres_sim, sim_idx = cosine_sim_matrics(tf_idf_mat)
#     print(f"\n COSINE-VECTOR- \n {genres_sim}")
#     print(f"\n SORTED-COSINE-VECTOR- \n {genres_sim}")
#     inputs_mv = input()
#     r = recommandation_mvs_top10(inputs_mv, sim_idx)
#     print(r)

def main2():
    drive.mount("/content/gdrive")
    #data = pd.read_csv("/content/gdrive/My Drive/mvs_final_csvfile.csv")
    data = pd.read_csv("recomm_data.csv")
    df = data.drop("Unnamed: 0", axis=1)
    inputs_mv = input("---찾고자 하는 영화를 입력해주세요!--- \n")
    input_method_of_recommandation = int(input('''
                    검색하시는 영화 이외에, 회원님이 좋아하실만한 영화 10개를 더 추천해드리려고 합니다! \n
                    1. 비슷하거나 같은 장르의 영화를 추천받고 싶으신가요? \n
                    2. 비슷하거나 같은 키워드나 주제를 가진 영화를 추천받고 싶으신가요? \n
                            '''))
    
    if input_method_of_recommandation == 1:
        tf_idf_mat = tf_idf_matrics(df["genres"])
    elif input_method_of_recommandation == 2:
        tf_idf_mat = tf_idf_matrics(df["keywords"])
    else:
        print("다시 선택해주세요!")

    #print(f'TF-IDF-VECTOR- \n {tf_idf_mat}')
    sim_idx = cosine_sim_matrics(tf_idf_mat)
    #print(f"\n COSINE-VECTOR- \n {genres_sim}")
    #print(f"\n SORTED-COSINE-VECTOR- \n {genres_sim}")
    print("---처리 중 입니다!--")
    recomm_df = recommandation_mvs_top10(inputs_mv, sim_idx, df)
    recomm_df = recomm_df.sort_values(by="year", ascending=False)
    recomm_df.to_csv("recommand_top_10_mvs.csv", index=False)

if __name__ == "__main__":
    main2()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
---찾고자 하는 영화를 입력해주세요!--- 
반도

                    검색하시는 영화 이외에, 회원님이 좋아하실만한 영화 10개를 더 추천해드리려고 합니다! 

                    1. 비슷하거나 같은 장르의 영화를 추천받고 싶으신가요? 

                    2. 비슷하거나 같은 키워드나 주제를 가진 영화를 추천받고 싶으신가요? 

                            2
---처리 중 입니다!--
[9]
[[   9    0  172 7290 3132 3135 2359 6867  286 6601]]
[   9    0  172 7290 3132 3135 2359 6867  286 6601]
